# NeuroHand: EEG Data Exploration

Этот notebook поможет вам понять структуру EEG данных и научиться их визуализировать.

**Цели:**
1. Загрузить preprocessed EEG данные
2. Визуализировать сырые сигналы
3. Анализировать частотный спектр
4. Понять различия между классами

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from src.visualization.plot_signals import plot_eeg_channels, plot_power_spectrum

%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

print("✅ Imports successful!")

## 1. Load Data

Загрузим preprocessed данные из `data/processed/` директории.

In [ ]:
# Load training data
data_dir = Path('../data/processed')

X_train = np.load(data_dir / 'train_data.npy')
y_train = np.load(data_dir / 'train_labels.npy')

print(f"Training data shape: {X_train.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"\nClass distribution: {np.bincount(y_train)}")
print(f"\nData range: [{X_train.min():.3f}, {X_train.max():.3f}]")

## 2. Visualize EEG Channels

Визуализируем все каналы одного trial.

In [ ]:
# Select a random trial
trial_idx = 0
signal = X_train[trial_idx]  # [channels, samples]
label = y_train[trial_idx]

class_names = ['Left Hand', 'Right Hand', 'Feet', 'Tongue']
print(f"Trial {trial_idx}: {class_names[label]}")

# Plot channels
fig = plot_eeg_channels(
    signal,
    sampling_rate=250,
    title=f"EEG Channels - {class_names[label]}"
)
plt.show()

## 3. Power Spectral Density

Анализируем частотный состав сигнала.

In [ ]:
# Plot PSD for channel 0
fig = plot_power_spectrum(
    signal,
    sampling_rate=250,
    channel_idx=0,
    title="Power Spectral Density - Channel 0"
)
plt.show()

## 4. Compare Classes

Сравним average сигналы для разных классов.

In [ ]:
# Calculate average signals per class
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
axes = axes.flatten()

for class_idx in range(4):
    # Get all trials for this class
    class_trials = X_train[y_train == class_idx]
    avg_signal = class_trials.mean(axis=0)
    
    # Plot first channel
    time = np.arange(avg_signal.shape[1]) / 250
    axes[class_idx].plot(time, avg_signal[0])
    axes[class_idx].set_title(f"{class_names[class_idx]} (n={len(class_trials)})")
    axes[class_idx].set_xlabel('Time (s)')
    axes[class_idx].set_ylabel('Amplitude')
    axes[class_idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Summary

**Что мы узнали:**
1. EEG данные имеют форму `[trials, channels, samples]`
2. Каждый trial - это 4 секунды записи (1000 samples at 250 Hz)
3. Сигналы normalized (mean ~0, std ~1)
4. Разные классы имеют subtle различия в паттернах

**Next Steps:**
- Experiment с разными каналами
- Analyze frequency bands (theta, alpha, beta)
- Try разные методы visualization